In [2]:
library(dbplyr)
library(DBI)
library(RSQLite)
library(dplyr)
library(SummarizedExperiment)
library(tidyr)

In [13]:
export.SummarizedExperiment_GEO <- function(GEOID, summary){
    ###  Check user input
    ### user input a GEOID
    # GEOID="GSE83894"  ### taking this GSE83894 as an example
    
    #connecting to SQL database in R
    con <- DBI::dbConnect(RSQLite::SQLite(), dbname = "../data/mprabase_v4_6.db")
    
    ## read from db to dataframe
    
    ### read data from selected GEO_number
    sqlStatement_seltable <- paste("SELECT * FROM datasets 
    INNER JOIN designed_library ON datasets.datasets_id = designed_library.datasets_id 
    INNER JOIN  sample ON designed_library.library_id=sample.library_id 
    INNER JOIN  library_sequence ON sample.library_id=library_sequence.library_id 
    INNER JOIN element_score ON library_sequence.library_element_id=element_score.library_element_id 
    WHERE datasets.GEO_number=","'",GEOID,"'",sep="")
    sel_table<- dbGetQuery(con, sqlStatement_seltable)
    
    #### generate coldata for grange from selected GEO_number
    sqlStatement_coldata  <- paste("SELECT datasets.PMID, datasets.GEO_number,datasets.labs FROM datasets
    WHERE datasets.GEO_number=","'",GEOID,"'",sep="")
    colData=  dbGetQuery(con,sqlStatement_coldata )
    
    ####remove duplicated record
    testing_colnames_1 <- sel_table[!duplicated(as.list(sel_table))]
    testing_colnames_2 <-  testing_colnames_1[!duplicated(testing_colnames_1$element_sample_id),]
    testing_colnames_2 <- subset(testing_colnames_2, select= -c(library_element_name.1, sample_id.1))
    coord_all_table <- separate(data = testing_colnames_2, col = element_coordinate, into= c("seqnames","start","end"))
    
    ##### export Grange
    gr = GRanges(seqnames = as.character(unlist(coord_all_table$seqnames)), 
    ranges = IRanges(as.numeric(unlist(coord_all_table$start)),
    end=as.numeric(unlist(coord_all_table$end)),
    names = unlist(coord_all_table$element_sample_id)))
    
    #### making ColData by dropping genomic-region columns

    #coord_all_table$start=as.numeric(coord_all_table$start)
    #coord_all_table$end=as.numeric(coord_all_table$end)
    
    mcols(gr) = subset(coord_all_table,select=-c(seqnames,start,end,sample_id,element_sample_id))
    
    #### making SummarizedExperiment
    SE1=SummarizedExperiment(assays=list(ratio=(as.matrix(coord_all_table$score))),rowRanges=gr,colData=colData)
    metadata(SE1)=summary
    return(SE1)
}

In [14]:
# load summary file
summary <- read.table("../inst/summary.csv", header=T, sep=',')

In [15]:
head(summary)

,PMID,GEO_number,SRP_number,labs,sample_name,number_of_elements,Library_strategy,Organism,Cell_line_tissue,DNA_RNA_reps
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,27831498,GSE83894,,"Shendure,Ahituv",Mutant_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
2,27831498,GSE83894,,"Shendure,Ahituv",Wild-type_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
3,30045748,GSE113978,,Shendure,Klein_STARR_Tiling_HepG2,5218,STARR-seq,Homo sapiens,HepG2,3
4,30045748,GSE113978,,Shendure,Klein_STARR_Orthologs_HepG2,5218,STARR-seq,Homo sapiens,HepG2,3
5,23892608,,SRP018414,"Ahituv,Ovcharenko,Shendure",Robin_MPRA_mouse,4966,plasmid,Mus musculus,mouse liver,3
6,23892608,,SRP018414,"Ahituv,Ovcharenko,Shendure",Robin_Hepg2,4966,plasmid,Homo sapiens,HepG2,3


In [16]:
filter(summary, GEO_number=="GSE83894")

PMID,GEO_number,SRP_number,labs,sample_name,number_of_elements,Library_strategy,Organism,Cell_line_tissue,DNA_RNA_reps
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
27831498,GSE83894,,"Shendure,Ahituv",Mutant_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
27831498,GSE83894,,"Shendure,Ahituv",Wild-type_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3


In [24]:
se <- export.SummarizedExperiment_GEO("GSE83894", filter(summary, GEO_number=="GSE83894"))

In [25]:
se

class: RangedSummarizedExperiment 
dim: 4880 1 
metadata(10): PMID GEO_number ... Cell_line_tissue DNA_RNA_reps
assays(1): ratio
rownames(4880): DS0001-SID01-ESID00000001 DS0001-SID01-ESID00000002 ...
  DS0001-SID02-ESID00002439 DS0001-SID02-ESID00002440
rowData names(24): datasets_id datasets_name ... sequence score
colnames: NULL
colData names(3): PMID GEO_number labs

In [26]:
head(assays(se)$ratio)

DS0001-SID01-ESID00000001,0.9859025
DS0001-SID01-ESID00000002,1.0603448
DS0001-SID01-ESID00000003,0.9077347
DS0001-SID01-ESID00000004,1.1769280
DS0001-SID01-ESID00000005,0.7886169
DS0001-SID01-ESID00000006,0.9539160


In [27]:
head(rowRanges(se))

GRanges object with 6 ranges and 24 metadata columns:
                            seqnames            ranges strand | datasets_id
                               <Rle>         <IRanges>  <Rle> | <character>
  DS0001-SID01-ESID00000001    chr10 11917871-11917984      * |      DS0001
  DS0001-SID01-ESID00000002    chr10 34165653-34165745      * |      DS0001
  DS0001-SID01-ESID00000003    chr10 52009954-52010059      * |      DS0001
  DS0001-SID01-ESID00000004    chr10 60767336-60767487      * |      DS0001
  DS0001-SID01-ESID00000005    chr10 60797400-60797480      * |      DS0001
  DS0001-SID01-ESID00000006    chr10 72112555-72112707      * |      DS0001
                               datasets_name        PMID  GEO_number
                                 <character> <character> <character>
  DS0001-SID01-ESID00000001 Inoue_MPRA_HepG2    27831498    GSE83894
  DS0001-SID01-ESID00000002 Inoue_MPRA_HepG2    27831498    GSE83894
  DS0001-SID01-ESID00000003 Inoue_MPRA_HepG2    27831498    GS

In [29]:
# TODO
colData(se)

DataFrame with 1 row and 3 columns
         PMID  GEO_number            labs
  <character> <character>     <character>
1    27831498    GSE83894 Shendure,Ahituv

In [31]:
metadata(se)

PMID,GEO_number,SRP_number,labs,sample_name,number_of_elements,Library_strategy,Organism,Cell_line_tissue,DNA_RNA_reps
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
27831498,GSE83894,,"Shendure,Ahituv",Mutant_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
27831498,GSE83894,,"Shendure,Ahituv",Wild-type_integrase_HepG2,2440,lentiMPRA,Homo sapiens,HepG2,3
